In [28]:
import pandas as pd 
credentials = pd.read_csv('credentials.csv')
consumer_key = credentials['consumer_key'][0]
consumer_secret = credentials['consumer_secret'][0]
access_token = credentials['access_token'][0]
access_secret = credentials['access_secret'][0]
 
import tweepy
from tweepy import OAuthHandler
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True)

import warnings
warnings.simplefilter('ignore')

In [26]:
tweepy.API?

In [9]:
from pymongo import MongoClient
client = MongoClient()
news_tweets = client.tweets_db.new_tweets


In [44]:
news_tweets.count()

24305

In [29]:
tweets = api.user_timeline(screen_name = "ABCnews",count=200)

In [9]:
tw = my_tweets[0]

In [34]:
users = api.friends('@amnasri2')

In [35]:
[u.screen_name for u in users]

[u'BNNBloomberg',
 u'TwitterNews',
 u'abcnews',
 u'ABC',
 u'CBCVancouver',
 u'CTVVancouver',
 u'GlobalBC',
 u'cbcnewsbc',
 u'AM730Traffic',
 u'CNN',
 u'NBCNews',
 u'BBCWorld']

In [38]:
import json

In [64]:
tweets = api.user_timeline(screen_name = "ABCnews",count=200)


In [15]:
users = api.friends('@amnasri2')

In [30]:
import time
def collect_tweets(users):
    curr_count = news_tweets.count()
    now = time.time()
    print("Started collecting tweets")
    for user in users:
        #tweets = api.user_timeline(screen_name = user.screen_name, count=200)
        tweets = tweepy.Cursor(api.user_timeline,screen_name=user.screen_name).items()
        for tw in tweets:

            if news_tweets.find_one({'id':tw.id}):
                continue

            try:
                news_tweets.insert_one(tw._json)
            except DuplicateKeyError:
                continue
                
    new_count = news_tweets.count()
    new_time = time.time() - now
    print("Collected %d tweets in %.2f seconds" % (new_count - curr_count, new_time))
    

In [ ]:
users = api.friends('@amnasri2')


In [31]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval):
        self._timer     = None
        self.interval   = interval
        self.is_running = False
        self.start()

    def _run(self):
        collect_tweets(users)
        self.is_running = False
        self.start()
        

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [46]:
interval = 30 * 60  # 30 minutes
rt = RepeatedTimer(interval)

In [45]:
rt.stop()

In [146]:
import threading
import time


class MyThread(threading.Thread):
    def __init__(self, interval):
        self.stop = False
        self.interval = interval
        super(MyThread, self).__init__()
        
    def run(self):
        collect_tweets(users)
        time.sleep(self.interval)
    
        
mythread = MyThread(10)  # ...Instantiate a thread and pass a unique ID to it
mythread.start()                                   # ...Start the thread


Started collecting tweets
Collected 13 tweets in 5.79 seconds


In [119]:
import threading
th = threading.Timer?


In [ ]:
th = threading.Timer

In [118]:
th.start()

hellow


In [110]:
news_tweets.count()

23324

In [89]:
#news_tweets.drop()

In [140]:
from scipy import sparse
from sklearn.base import BaseEstimator, TransformerMixin
import re
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        comments_clean = []
        for c in X:
            c = c.replace('\\\\', '\\')
            c = c.replace('\\n', ' ')
            #c = c.replace('\'', '')
            c = re.sub(r'[-_"]', '', c)
            c = re.sub(r'[*%&,?!;]', ' ', c)
            c = re.sub(r"(.)\1{2,}", '\g<1>', c)
            c = re.sub(r'\.(\s+|$)', ' ', c)
            c = re.sub(r'[^\x00-\x7F]+',' ', c)
            #c = re.sub(r'@[^ ]*', ' ', c)
            #c = re.sub(r'[0-9]+', ' ', c)
            c = [wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(w, pos='v')).lower() 
                          for w in c.split()]
            c = " ".join(w for w in c if len(w) > 2)
            #c = " ".join([w.lower() for w in c.split()])# if 3<=len(w)<=40])
            comments_clean.append(c)
        return comments_clean

prep = Preprocessor()

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english', token_pattern=u'((?u)\\b\\w\\w+\\b|[#@]\w+)'
                             )

documents = [tw['text'] for tw in news_tweets.find()]
documents = prep.fit_transform(documents)
#tweets_text_vec = tfidv_vectorizer.fit_transform(text_processed)
#tweets_text_count_vec = count_vectorizer.fit_transform(text_processed)

In [137]:


import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def lemmatize_stemming(text, stem=True):
    result = lemmatizer.lemmatize(text, pos='v')
    if stem:
        result = stemmer.stem(result)
    return result
    
def preprocess(text):
    
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
            and len(token) > 3 and len(token) < 40 \
            and token.isalpha:
            result.append(lemmatize_stemming(token, stem=False))
    return " ".join(result)


from sklearn.datasets import fetch_20newsgroups

#dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = [preprocess(tw['text']) for tw in news_tweets.find()]


In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


no_features = 1000

# Data cleaning

import re
from nltk.corpus import stopwords


# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
#tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 10

# Run NMF
#nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=15, learning_method='online', learning_decay=.9, learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
trump say fund court analysis govt rule price supreme president
Topic 1:
india look rain group weather life help amp start forecast
Topic 2:
trump president watch live shoot north say summit kim man
Topic 3:
bank http know deal need best open investment west chinese
Topic 4:
story kill snow growth read right hour economy line black
Topic 5:
year come week financial just break chief times june biggest
Topic 6:
new face high job crisis york market say force power
Topic 7:
world cup today south win watch 2018 woman #worldcup home
Topic 8:
china trade end warn brexit hit global way month use
Topic 9:
trump house plan say child family government border make want


In [150]:
documents

[u'torstar cut fulltime parttime staff starmetro toronto https://t.co/6bekb8cw8j https://t.co/vdgi141qyl',
 u'top pick from john zechner zechner associates https://t.co/v63zsa6vjf https://t.co/s7szsjlsh6',
 u'gas tax increase 1.5 cent for metro vancouver pay for transit https://t.co/txddi0no5k https://t.co/nasrguiscg',
 u'health minister call for more cannabis pharmacare support winnipeg https://t.co/dj7a3ceqbv https://t.co/imujkujnq6',
 u'millions u.s adidas customer alert data breach https://t.co/igld5cyqmi https://t.co/6hsfzksqmj',
 u"will thrash the economy like 'tsunami': allstate ceo https://t.co/xfmial3mf4 https://t.co/xowqrpczdd",
 u'nike see first north american sale gain quarter https://t.co/mecgaxpakx https://t.co/qjbfeogp5a',
 u"canadian consumer 'driving without spare tire' face recession say pimco devlin https://t.co/wntqpvdggb https://t.co/rxx0gi2fed",
 u"don't worry about recession until 2019: gmp securities https://t.co/0vardgtwhz @catherinebiztv https://t.co/dvufa4ssa

In [149]:
lda.aa

(10, 10000)